In [597]:
import matplotlib.pyplot as plt
import numpy as np
import sys

MTOW = 15


# CONSTANTS
AIR_DEN = 1.25
GRAVITY = 9.82

MAGIC_NUMBER = 0.6171062

ESC_EFF = 0.95

prop1 = np.genfromtxt("0.291667.csv", delimiter=",")
prop2 = np.genfromtxt("0.416667.csv", delimiter=",")
prop3 = np.genfromtxt("0.625.csv", delimiter=",")
prop4 = np.genfromtxt("1.csv", delimiter=",")

prop_pd = np.array([0.21667, 0.416667, 0.625, 1])

prop_data = [
    prop1,
    prop2,
    prop3,
    prop4
]


def calculate_score(takeoff_dist,
                    max_speed, 
                    min_speed, 
                    flight_distance,
                    battery_capacity,
                    payload_mass,
                    payload_volume,
                    landing_angle=5, 
                    landing_dist=10,
                    climb_angle=20, 
                    glide_ratio=3, ):

    payload_energy_consumed = (battery_capacity/(4.2*6))/(flight_distance*payload_mass)

    reference_airplane = np.array([
        30,		# Take-off Distance
        12,		# Climb Angle
        3,		# Glide Ratio
        30,		# Max Speed
        10,		# Min Speed
        5,		# Landing Angle
        50,		# Landing Distance
        15,		# Flight Distance
        0.4,	# Specific Energy Consumtion
        5,		# Payload Mass
        0.5		# Payload Volume
	])
    
    our_plane = np.array([
        takeoff_dist,		# Take-off Distance
        climb_angle,		# Climb Angle
        glide_ratio,		# Glide Ratio
        max_speed,		# Max Speed
        min_speed,		# Min Speed
        landing_angle,		# Landing Angle
        landing_dist,		# Landing Distance
        flight_distance,		# Flight Distance
        payload_energy_consumed,	# Specific Energy Consumtion
    	payload_mass,		# Payload Mass
        payload_volume		# Payload Volume
	])
    
    weights = np.array([
        -4,
        7,
        25,
        4,
        -4,
        5,
        -3,
        8,
        -12,
        10,
        18
	])
    div = our_plane/reference_airplane
    scores = weights*np.tanh(div - 1)
    score = np.sum(scores)
    
    print("RESULTS")
    print("=======================")
    
    print("Take off Distance: ", takeoff_dist)
    print("Climb Angle: ", climb_angle)
    print("Glide ratio: ", glide_ratio)
    print("Max speed: ", max_speed)
    print("Minimum speed: ", min_speed)
    print("Landing Angle: ", landing_angle)
    print("Landing Distance: ", landing_dist)
    print("Payload Energy Consumed: ", payload_energy_consumed)
    print("Payload Mass: ", payload_mass)
    print("Payload Volume: ",payload_volume )

    #print("Div:", div)
    #print("Scores: ", scores)
    
    print("=====================")
    print("SCORES")
    
    print("Take off Distance: ", scores[0])
    print("Climb angle: ", scores[1])
    print("Glide Ratio ", scores[2])
    print("Max Speed: ", scores[3])
    print("Min Speed: ", scores[4])
    print("Landing Angle: ", scores[5])
    print("Landing Distance: ", scores[6])
    print("Flight Distanace: ", scores[7])
    print("Specific Energy Consumtion: ", scores[8])
    print("Payload Mass: ", scores[9])
    print("Payload Volume: ", scores[10])

    print(score)
    


In [ ]:
def calc_electric_power(motor, rpm, power):
    """
    This function does not take into account KV reduction under load, therefor propeller will spin at approx 80% RPM at max load, possibly even 60%
    
    Therefor its good if motor is selected which is a bit overrated
    
    """
    #print("in power: ", power)
    torque = power/(2*np.pi*rpm/60)
    
    #print("torque", torque)
    
    torque_constant = 60/(2*np.pi*motor['kv'])
    #print("torque const:", torque_constant)
    motor_current = motor['idle_current'] + torque/torque_constant
    #print("motor current", motor_current)
    power = (((rpm/motor['kv']) + (motor['resistance']*motor_current)) * motor_current)/ESC_EFF
    
    #print("Power:!!", power)
    
    return power

In [ ]:
def prop_pt_calc(motor, prop_matrix, airspeed, diameter, max_power):
    max_rpm = 60/(np.pi*(diameter)*0.00291545)
    
    for current_test_rpm in range(100, int(max_rpm), 100):
        rps = current_test_rpm/60
        #print("rpm: ", current_test_rpm)
        
        advance_ratio = airspeed/(current_test_rpm*diameter)
        
        idx = np.abs(prop_matrix[:,0] - advance_ratio).argmin()

        coeff_thrust = prop_matrix[idx, 1]
        coeff_power = prop_matrix[idx, 2]
        
        #print("Coeff thrust: ", coeff_thrust)
        #print("Coeff power: ", coeff_power)
        #print(diameter)

        thrust = coeff_thrust*AIR_DEN*np.power(rps, 2)*np.power(diameter, 4)
        power = coeff_power*AIR_DEN*np.power(rps, 3)*np.power(diameter, 5)
        
        power = calc_electric_power(motor, rpm=rps*60, power=power)
        
        #print("thrust: ", thrust)
        #print("power: ", power)
        
        if max_power < power:
            power = coeff_power*AIR_DEN*np.power(rps-100/60, 3)*np.power(diameter, 5)
            power = calc_electric_power(motor, rpm=(rps-100/60)*60, power=power)
            thrust = coeff_thrust*AIR_DEN*np.power(rps-100/60, 2)*np.power(diameter, 4)
            #print("Reached Max power")
            return power, thrust

        
    power = coeff_power*AIR_DEN*np.power(max_rpm/60, 3)*np.power(diameter, 5)
    thrust = coeff_thrust*AIR_DEN*np.power(max_rpm/60, 2)*np.power(diameter, 4)
    
    return power*2, thrust*2

    

In [ ]:
def take_off(plane_configuration, power):
    stall_velocity = (2*MTOW*GRAVITY/(plane_configuration['take_off_cl']*AIR_DEN*plane_configuration['wing_area']))**0.5

    dt = 0.001
    current_velocity = 0
    current_time = 0
    current_energy = 0
    current_distance = 0

    while current_velocity < stall_velocity:
        take_off_power, thrust = prop_pt_calc(
            motor=plane_configuration['motor'],
            prop_matrix=plane_configuration['prop_matrix'],
            airspeed=current_velocity, 
            diameter=plane_configuration['prop_diameter'],
            max_power=power
        )
        
        #print("Take off power: ", take_off_power)
        #print("Take off thrust: ", thrust)
        
        current_drag = 0.5*plane_configuration['take_off_cd']*AIR_DEN*(plane_configuration['fusilage_area'] + plane_configuration['wing_area'])*current_velocity**2
        
        #print("Current Drag:", current_drag)
        
        current_accel = (thrust - current_drag)/MTOW
        
        current_distance = current_distance + current_velocity*dt + 0.5*current_accel*dt**2
        current_velocity = current_velocity + current_accel*dt
        
        #print("Current velocity: ", current_velocity)
        
        current_energy = current_energy + take_off_power*dt
        
        current_time = current_time+dt
        
    takeoff_time = current_time
    takeoff_energy = current_energy/3600
    takeoff_distance = current_distance

    print(f"Takeoff time: {round(takeoff_time, 1)} s")
    print(f"Energy: {round(takeoff_energy, 1)} Wh")
    print(f"Takeoff distance: {round(takeoff_distance, 1)} m")

    return takeoff_distance, takeoff_energy, stall_velocity

In [607]:
# TODO

def climb():
    drag = 0.5*cruise_cd*AIR_DEN*(fusilage_area + wing_area)*cruise_airspeed**2
    lift = 0.5*cruise_cl*AIR_DEN*(wing_area)*cruise_airspeed**2

In [602]:
def power_from_drag(motor, prop_matrix, airspeed, diameter, drag):
    current_difference = 100000000000000
    coeff_power = 0
    drag = drag/2 # Two propellors
    
    # Solve for rps
    for idx, advance_ratio in enumerate(prop_matrix[:len(prop_matrix[:,0])-1,0]):
        coeff_thrust = prop_matrix[idx, 1]
        #print("IDX: ", idx)
        
        rps_from_advance = airspeed/(advance_ratio*diameter)
        #print("RPM from advance: ", rps_from_advance*60)
        
        rps_from_thrust = np.pow(drag/(coeff_thrust*AIR_DEN*np.pow(diameter, 4)), 1/2)
        #print("RPM from thrust: ", rps_from_thrust*60)
       
        if np.abs(rps_from_thrust - rps_from_advance) < current_difference:
            current_difference = np.abs(rps_from_thrust - rps_from_advance)
            
            current_rps = (rps_from_thrust + rps_from_advance)/2
            
            coeff_power = prop_matrix[idx, 2]
            
            selected_idx = idx
            

            #print("Current RPM: ", current_rps*60)
            #print("Current Difference: ", current_difference)
            #print("Coeff power: ", coeff_power)
        
        
    if idx <= 0 or idx >= len(prop_matrix[:,0]):
        return False # Exit early if error

    
    #print(f"Advance Ratio: {round(advance_ratio,1)}")
    #print(f"Coeff Thrust: {round(prop_matrix[selected_idx, 2],1)}")
    #print(f"Coeff Power: {round(coeff_power,1)}")
    
    #print(f"Prop eff: {prop_matrix[selected_idx,3]}")
    #print("Test: ", prop_matrix[selected_idx,1]*prop_matrix[selected_idx,0]/prop_matrix[selected_idx,2])

    
    #print(f"Cruise RPM: {round(current_rps*60)}")
    
    power = coeff_power*AIR_DEN*np.pow(current_rps, 3)*np.pow(diameter, 5)
    #print("power1: ",power)
    #                   Voltage                             Voltage                  Current
    power = calc_electric_power(motor, rpm=current_rps*60, power=power)
    #print("power2:", power)
    return power*2 # Two propellors

In [603]:
def endurance(plane_configuration, cruise_speed, energy_left):
    area_drag = plane_configuration['wing_area'] + plane_configuration['fusilage_area']+plane_configuration['tail_area']
    cruise_drag = 0.5*AIR_DEN*plane_configuration['cruise_cd']*area_drag*cruise_speed**2

    power = power_from_drag(
        motor=plane_configuration['motor'],
        prop_matrix=plane_configuration['prop_matrix'],
        airspeed=cruise_speed,
        diameter=plane_configuration['prop_diameter'],
        drag=cruise_drag)


    print("Energy left: ", energy_left)
    cruise_time = energy_left/power
    cruise_time = cruise_time*3600
    # Assuming airspeed = ground speed
    cruise_distance = cruise_speed*cruise_time
    cruise_distance = cruise_distance/1000
    print(f"Cruise power: {round(power, 1)}")
    print(f"Cruise drag: {round(cruise_drag, 1)}")
    print(f"Cruise Time: {round(cruise_time/3600, 1)} h")
    print(f"Cruise Distance: {round(cruise_distance, 1)}km")
    return cruise_distance


In [604]:

def generate_prop_matrix(diameter, pitch):
    pd = pitch/diameter
    
    print("PD: ", pd)
    pd_upperbound = 0
    pd_lowerbound = 0

    if pd >= prop_pd.max() or pd < prop_pd.min():
        print("Incorrect PD value")
        return False

    # Find upper and lower bound of P/D
    for idx, i in enumerate(prop_pd):
        if pd < i:
            pd_upperbound = i
            #print(idx_lowerbound)
            break

    for i in reversed(prop_pd):
        if pd >= i:
            pd_lowerbound = i
            break

    #print("PD Upperbound: ", pd_upperbound)
    #print("PD Lowerbound: ", pd_lowerbound)
    #print(idx)
    m, b = np.polyfit([pd_lowerbound, pd_upperbound], [0, 1], 1)
    moj = m*pd+b
    
    # Generate prop parameter matrix by interpolation
    new_matrix = (1-moj)*prop_data[idx-1] + moj*prop_data[idx]
    return new_matrix



In [605]:

def calc_max_speed(plane_configuration, max_power, prop_matrix, diameter, cruise_airspeed):
    max_rpm = 60/(np.pi*(diameter)*0.00291545)
    area_drag = plane_configuration['wing_area'] + plane_configuration['fusilage_area']
    
    # Find max speed from thrust
    for airspeed in range(cruise_airspeed, 100):
        drag = 0.5*AIR_DEN*plane_configuration['cruise_cd']*area_drag*airspeed**2
        
        # Reach max power from rpm
        for rpm in range(100, int(max_rpm), 100):
            power = prop_matrix[24,2]*AIR_DEN*np.pow(rpm/60, 3)*np.pow(diameter, 5)
            #print("Power test: ", power)
            #print("RPS test: ", rpm)
            
            if power < max_power:
                continue
            break
            
        thrust = prop_matrix[24,1]*AIR_DEN*np.pow(rpm/60, 2)*np.pow(diameter, 4)
        
        #print("RPM: ", rpm)
        #print("Thrust: ", thrust)
        #print("Power: ", power)
        #print("Drag: ", drag)
        
        if thrust > drag:
            continue
        
        else:
            print("Max speed", airspeed)
            return airspeed



In [606]:
# Energy storage in Wh, assuming 260 Wh/kg and 3 kg of batteries

motor ={
    "name": "P60 KV340",
    "kv": 340,
    "max_power": 1632,
    "resistance": 0.035,
    "weight": 0.345,
    "idle_current": 2
}

landing_gear_weight = 0.95
motor_weight = 0.15
prop_weight = 0.05

battery_weight = 0.5
battery_energy = 360*battery_weight
max_available_electric_power = 54*2*3.4*6


max_available_mechanical_power = max_available_electric_power*ESC_EFF

print("Max available electric power: ", max_available_electric_power)

main_body = 5
fligtcontroller = 0.07

payload = 15 - battery_weight - landing_gear_weight - motor['weight']*2 - prop_weight*2 - main_body - fligtcontroller
payload_vol = 0.5
# Needs a better estimation of Cd_fuse!!! Scaled with fuselage surface area
fusilage_cd = 0.02

#----- Optim
cruise_airspeed = 15 # m/s
wing_area = 2 # m^2
fusilage_area = 1 # m^2

diameter = 20 # inches
diameter = diameter*25.4/1000
pitch = 12 # inches
pitch = pitch*25.4/1000
p_d = diameter/pitch
prop_matrix = generate_prop_matrix(diameter, pitch)

if type(prop_matrix) == bool:
    print("Something went wrong")
    sys.exit("prop matrix wrong")

take_off_cd = 0.13
take_off_cl = 1.3

cruise_cd = 0.01
take_off_cd = fusilage_cd + take_off_cd #+ tail_cd

plane_configuration = {
    "take_off_cl": take_off_cl,
    "take_off_cd": take_off_cd,
    "cruise_cd": cruise_cd,
    "prop_matrix": prop_matrix,
    "prop_diameter": diameter,
    "wing_area": wing_area,
    "fusilage_area": fusilage_area,
    "tail_area": 0.5,
    "payload_weight": payload,
    "payload_vol": payload_vol,
    "battery_energy": battery_energy,
    "motor": motor
}

max_speed = calc_max_speed(plane_configuration, max_available_electric_power, prop_matrix, diameter, cruise_airspeed)

take_off_dist, take_off_energy, stall_velocity = take_off(
    plane_configuration=plane_configuration,
    power=max_available_electric_power
    )

endurance_energy = battery_energy - take_off_energy

cruise_distance = endurance(
    plane_configuration=plane_configuration,
    cruise_speed=cruise_airspeed,
    energy_left=endurance_energy, 
)


calculate_score(
    takeoff_dist=take_off_dist,
    max_speed=max_speed,
    min_speed=stall_velocity,
    flight_distance=cruise_distance,
    battery_capacity=battery_energy,
    payload_mass=payload,
    payload_volume=payload_vol
)

Max available electric power:  2203.2
PD:  0.5999999999999999
Max speed 36
Takeoff time: 1.6 s
Energy: 1.0 Wh
Takeoff distance: 8.3 m
Energy left:  178.99739234703256
Cruise power: 150.4
Cruise drag: 4.9
Cruise Time: 1.2 h
Cruise Distance: 64.3km
RESULTS
Take off Distance:  8.255615033783632
Climb Angle:  20
Glide ratio:  3
Max speed:  36
Minimum speed:  9.520827371933274
Landing Angle:  5
Landing Distance:  10
Payload Energy Consumed:  0.014452907357357804
Payload Mass:  7.690000000000001
Payload Volume:  0.5
SCORES
Take off Distance:  2.479526533513422
Climb angle:  4.079480617435371
Glide Ratio  0.0
Max Speed:  0.7895012808996158
Min Speed:  0.1915224910222708
Landing Angle:  0.0
Landing Distance:  1.9921103108035472
Flight Distanace:  7.977579818805794
Specific Energy Consumtion:  8.951965391668182
Payload Mass:  4.914725488009013
Payload Volume:  0.0
31.376411932157218
